DONE
Step 1: Preprocess the data - use the same function across the board
Clearing out the capitalization
Clearing out em dashes, symbols
Clearing out names

TO DO
Step 2: Implementing the features
- Goal: Train a logistic regression on 2 feature representations

Step 3: Implementing the regression based on that data

# Part 1: Imports and Cleaning Text

In [16]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import numpy as np
from sklearn.linear_model import LogisticRegression


In [2]:
# Read in data
train = pd.read_csv('data/train.csv')
val = pd.read_csv('data/val.csv')

In [3]:
# Helper function for cleaning text
def clean_html(text):
    if pd.isna(text):
        return text
    # Remove HTML tags
    clean = re.sub(r'<.*?>', '', str(text))
    # Remove extra whitespaces
    clean = re.sub(r'\s+', ' ', clean).strip()
    # Replace HTML entities
    clean = re.sub(r'&amp;', '&', clean)
    clean = re.sub(r'&lt;', '<', clean)
    clean = re.sub(r'&gt;', '>', clean)
    clean = re.sub(r'&quot;|&#34;', '"', clean)
    clean = re.sub(r'&apos;|&#39;', "'", clean)
    return clean

In [5]:
# use the clean_html function to clean the training data
train['snip'] = train['snip'].apply(clean_html)
val['snip'] = val['snip'].apply(clean_html)

print(train)
print(val)

                                                    snip   channel
0      first of all, it feels like covid again but in...  FOXNEWSW
1      to be a software drivenrganization where softw...     CSPAN
2      you discuss the power of ai to revolutionize t...    CSPAN2
3      ai bots like chatgpt and google's bard gained ...   BBCNEWS
4      . >> i could sleep ten hours ai night if i was...  FOXNEWSW
...                                                  ...       ...
19868  cardiovascular science, but they're also pione...  FOXNEWSW
19869  i of ai in different fields. have of ai in dif...   BBCNEWS
19870  weighing down on the major averages, both tech...      KTVU
19871  i also think crypto ai that legislation be fro...    CSPAN2
19872  as we have worked to monitor the adoption iden...    CSPAN2

[19873 rows x 2 columns]
                                                   snip    channel
0     . ♪ >> there's a kyu cho right have things tha...  BLOOMBERG
1     he says the ai tool helped cre

In [12]:
# evaluation metric equation
def eval(y_pred, y_true):
    correct = (y_pred == y_true)   # Boolean array: True if correct, False if wrong
    accuracy = correct.sum() / len(y_true)  # Correct / Total
    return accuracy

In [7]:
# Initialize and fit TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(train['snip'])

# Calculate word complexity per snip
word_complexity = X_tfidf.sum(axis=1) / (X_tfidf != 0).sum(axis=1)
word_complexity = np.array(word_complexity).flatten()

# Add it to the train dataframe
train['word_complexity'] = word_complexity

print(train)

                                                    snip   channel  \
0      first of all, it feels like covid again but in...  FOXNEWSW   
1      to be a software drivenrganization where softw...     CSPAN   
2      you discuss the power of ai to revolutionize t...    CSPAN2   
3      ai bots like chatgpt and google's bard gained ...   BBCNEWS   
4      . >> i could sleep ten hours ai night if i was...  FOXNEWSW   
...                                                  ...       ...   
19868  cardiovascular science, but they're also pione...  FOXNEWSW   
19869  i of ai in different fields. have of ai in dif...   BBCNEWS   
19870  weighing down on the major averages, both tech...      KTVU   
19871  i also think crypto ai that legislation be fro...    CSPAN2   
19872  as we have worked to monitor the adoption iden...    CSPAN2   

       word_complexity  
0             0.057407  
1             0.079768  
2             0.076151  
3             0.077301  
4             0.075782  
...      

In [8]:
# Transform validation snips using the same TF-IDF vectorizer
X_val_tfidf = vectorizer.transform(val['snip'])

# Compute complexity
val_word_complexity = X_val_tfidf.sum(axis=1) / (X_val_tfidf != 0).sum(axis=1)
val_word_complexity = np.array(val_word_complexity).flatten()

# Add to val DataFrame
val['word_complexity'] = val_word_complexity

print(val)

                                                   snip    channel  \
0     . ♪ >> there's a kyu cho right have things tha...  BLOOMBERG   
1     he says the ai tool helped create a new fronti...       KPIX   
2     . >> the all new godaddy arrow put your busine...       CNNW   
3     in some cases they are powered by generative a...      CSPAN   
4     this was a ivotal it comes to ai. this was a p...    BBCNEWS   
...                                                 ...        ...   
3034  however, the ai trade is only one part of the ...       CNBC   
3035  oz but also was highlighted as a product by cr...     CSPAN2   
3036  the all new godaddy airo helps you get your bu...       CNBC   
3037  we are going to be way ahead on ai. we have to...       CNBC   
3038  his fourth management role after spells at der...    BBCNEWS   

      word_complexity  
0            0.088216  
1            0.081808  
2            0.074722  
3            0.082919  
4            0.083567  
...            

In [13]:
# X = features, y = labels
x_train = train[['word_complexity']]  # Needs to be 2D
y_train = train['channel']

x_val = val[['word_complexity']]
y_val = val['channel']

In [17]:
# Train
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

# Predict
val_preds = model.predict(x_val)
print(val_preds)

# Evaluate
val_accuracy = eval(val_preds, y_val)
print(val_accuracy)

['CNNW' 'CNNW' 'CNNW' ... 'CNNW' 'CNNW' 'CNNW']
0.07535373478117802
